In [ ]:
%matplotlib inline
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.optimize import curve_fit
import scipy.optimize as opt
import matplotlib.lines as mlines
import pandas as pd
import csv
from skimage import io
import sled_image_tools
from matplotlib.gridspec import GridSpec
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.rcParams.update({'font.size': 24, 'font.family':'sans-serif', 'font.sans-serif':'Arial', 'pdf.fonttype':42})

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return (idx,array[int(idx)])

def twoD_Gaussian(xy, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    x, y = xy
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

def gauss (x,a,b,c):
    return a*np.exp(-(x-b)**2/(2*c**2))

x_pixels = np.linspace(0,1440, 1440)
y_pixels = np.linspace(0,1080, 1080)

# factory settings
freq_dir = 'Factory_Settings/Stroboscopy_Camera_Images/100kHz_31.2C_218Vdc_270Vpp/'
saveTag='_fact'
# freq_dir = 'Stroboscopy_V2/Factory_Settings/Stroboscopy_Camera_Images/160kHz_31.2C_218Vdc_270Vpp/' saveTag='_fact'
# freq_dir = 'Stroboscopy_V2/Factory_Settings/Stroboscopy_Camera_Images/220kHz_31.2C_218Vdc_270Vpp/' saveTag='_fact'
# freq_dir = 'Stroboscopy_V2/Factory_Settings/Stroboscopy_Camera_Images/280kHz_31.2C_218V_270Vpp/'
# saveTag='_fact'



# optimized settings

freq_dir  = 'Optimized_Settings/Stroboscopy_Camera_Images/100kHz_31.2C_218Vdc_270Vpp/'
saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/160kHz_31.0C_230Vdc_278Vpp/' saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/220kHz_30.2C_224Vdc_268Vpp/' saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/280kHz_26C_249Vdc_194Vpp/'
# saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/340kHz_24.5C_207Vdc_176Vpp/' saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/400kHz_24.4C_226Vdc_142Vpp/' saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/460kHz_25C_265VDC_132Vpp/' saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/460kHz_25C_293Vdc_224Vpp/' saveTag='_opt'
# freq_dir  = 'Stroboscopy_V2/Optimized_Settings/Stroboscopy_Camera_Images/520kHz_26C_276Vdc_134Vpp/' saveTag='_opt'

# saveTag='_opt'




data_dir = 'K:/Farinella_2024_repo/raw_data/Fig4/' # filepath has to be changed dep. on where dropbox is
print(data_dir+freq_dir)
file_list = glob.glob(data_dir+freq_dir+'*tif')
saveName=os.path.basename(file_list[0])
saveName = saveName[0:len(saveName)-4]+saveTag
image = np.zeros((len(file_list),len(y_pixels),len(x_pixels)))
print(file_list)
for k in range(len(file_list)):
    image[k,:,:] = io.imread(file_list[k])
    file_list[k]
# determine the angle of the scan
print(image.shape)

In [ ]:
image_name = image[0]

y0LHS = np.where(image_name == np.max(image_name))[0][0]
x0LHS = np.where(image_name == np.max(image_name))[1][0]

image_name = image[-1]

y0RHS = np.where(image_name == np.max(image_name))[0][0]
x0RHS = np.where(image_name == np.max(image_name))[1][0]

image_name = sum(image[:])# image[0]+image[-1]

x0LHS = 0
x0RHS = 1300

y0LHS = 350
y0RHS = 700

cmax=350

thresh = np.max(image_name)*0.0
masked_image = np.copy(image_name)
masked_image[masked_image < thresh] = 0

x_pixels = np.linspace(0,1440, 1440)
y_pixels = np.linspace(0,1080, 1080)

##########################################################################################

fig, ax0 = plt.subplots(figsize=(12,12))
gs = GridSpec(120,120)#,bottom=0.18,left=0.18,right=0.88)

ax0 = plt.subplot(gs[0:110,0:110])

# h0 = ax0.pcolor(masked_image,cmap='inferno')
h0 = ax0.imshow(masked_image,cmap='gray',vmin=0,vmax=cmax,interpolation='none')
# ax0.set_xlim(x0LHS-100,x0RHS+100)
# ax0.set_ylim(y0LHS-100,y0RHS+100)
ax0.set_xlim(x0LHS,x0RHS)
ax0.set_ylim(y0LHS,y0RHS)
ax0.set_aspect('equal')
ax0.set_yticks([])
ax0.set_xticks([])
sled_image_tools.imshow_cleanup(ax0)
cbaxes = fig.add_subplot(gs[0:110,115:120])
cbaxes.tick_params(labelsize=18)
cb = fig.colorbar(h0, cax = cbaxes, orientation='vertical')
cb.set_label('intensity (A.U.)',fontsize=20)
cbtext = cb.ax.yaxis.get_offset_text()

with PdfPages('K:/Farinella_2024_repo/processed_data/Fig4/'+saveName+'.pdf') as pdf:
    pdf.savefig(fig,bbox_inches='tight',pad_inches=0.05,dpi=600)  # saves the current figure into a pdf page
curr_ylim=list(ax0.axes.get_ylim())
curr_xlim=list(ax0.axes.get_xlim())
curr_aspect = (curr_ylim[1]-curr_ylim[0])/(curr_xlim[1]-curr_xlim[0])
print(np.nanmax(masked_image))
print(curr_ylim)
print(curr_xlim)
print(curr_aspect)
